In [2]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# import necessary libraries
import numpy as np
import pandas as pd
import xarray as xr

In [3]:
# Open the dataset
path = "/home/jatin/Downloads/sst1970-1979.nc"
ds = xr.open_dataset(path)

print(ds)

<xarray.Dataset>
Dimensions:  (lat: 180, lon: 360, time: 120)
Coordinates:
  * lon      (lon) float32 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * lat      (lat) float32 89.5 88.5 87.5 86.5 85.5 ... -86.5 -87.5 -88.5 -89.5
  * time     (time) datetime64[ns] 1970-01-01T12:00:00 ... 1979-12-01T12:00:00
Data variables:
    sst      (time, lat, lon) float32 ...
Attributes:
    Conventions:                CF-1.0
    title:                      Monthly version of HadISST sea surface temper...
    institution:                Met Office, Hadley Centre for Climate Research
    source:                     HadISST
    history:                    09/11/2006 HadISST converted to NetCDF from p...
    references:                 Rayner, N. A., Parker, D. E., Horton, E. B., ...
    comment:                    Data restrictions: for academic research use ...
    supplementary_information:  Updates and supplementary information will be...
    contact:                    john.kennedy@metoffice.g

In [4]:
# Extract the data variable
sst = ds['sst']
print(sst)

<xarray.DataArray 'sst' (time: 120, lat: 180, lon: 360)>
[7776000 values with dtype=float32]
Coordinates:
  * lon      (lon) float32 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * lat      (lat) float32 89.5 88.5 87.5 86.5 85.5 ... -86.5 -87.5 -88.5 -89.5
  * time     (time) datetime64[ns] 1970-01-01T12:00:00 ... 1979-12-01T12:00:00
Attributes:
    long_name:     Monthly 1 degree resolution SST
    units:         degrees C
    actual_range:  [-1000.         33.53844]
    description:   HadISST sea surface temperature. values of -1000 indicate ...


In [8]:
def month_to_season12(xMon):
    
    # xMon is the DataArray passed as an argument
    
    # Create a new array of seasons
    season = np.array(["DJF","JFM","FMA","MAM","AMJ","MJJ","JJA","JAS","ASO","SON","OND","NDJ"])
    
    # Get the rank of dataarray
    len_of_dim = xMon.sizes
    rank_of_dim = len(len_of_dim)
    
    # Check if the rank of dim is valid or invalid; if invalid, exit the function
    if (rank_of_dim == 2 or rank_of_dim >= 5):
        print("month_to_season12: rank = {}".format(rank_of_dim))
        print("----- rank currently not handled -----")
        return None
    
    # Check if number of months are multiple of 12; if not, exit the function
    no_of_months = 12
    no_of_time = len_of_dim[xMon.dims[0]]
    if ((no_of_time % no_of_months) != 0):
        print("month_to_season12: dimension must be a multiple of 12")
        return None
    no_of_years = no_of_time/no_of_months
    
    # Store the size of latitude and longitude
    if (rank_of_dim >= 3):
        size_of_lat = len_of_dim[xMon.dims[rank_of_dim-2]]
        size_of_lon = len_of_dim[xMon.dims[rank_of_dim-1]]
    
    # Check if dimensions are named or not; if unnamed, exit the function
    for i in range(0, rank_of_dim):
        if xMon.dims[i] == None or xMon.dims[i] == "":
            print("mon_to_season12: All dimensions must be named")
            print("\t\tdimension {} is missing".format(i))
            return None
    
    # Calculating seasonal mean for rank = 1
    if (rank_of_dim == 1):
        
        # Calculate seasonal mean for each season
        dr = xMon.rolling(time = 3, center = True).mean(skipna = True)
        dr[0] = (xMon[0] + xMon[1]) * 0.5
        dr[(no_of_time-1)] = (xMon[(no_of_time-2)] + xMon[(no_of_time-1)]) * 0.5
        
        # Create a new DataArray using the existing ones
        xSea = xMon.copy(data = dr)
        
        # Add and update the attributes
        xSea.attrs['season'] = season
        xSea.attrs['long_name'] = "seasonal means: " + xSea.attrs['long_name']
        xSea = xSea.rename("xSea")
        
        # Return the newly created DataArray
        return xSea
    
    
    # Calculating seasonal mean for rank = 3
    if (rank_of_dim == 3):
        
        # Calculate seasonal mean for each season
        dr = xMon.rolling(time = 3, center = True).mean(skipna = True)
        dr[0,:,:] = (xMon[0,:,:] + xMon[1,:,:]) * 0.5
        dr[(no_of_time-1),:,:] = (xMon[(no_of_time-2),:,:] + xMon[(no_of_time-1),:,:]) * 0.5
        
        # Create a new DataArray using the existing ones
        xSea = xMon.copy(data = dr)
        
        # Add and update the attributes
        xSea.attrs['season'] = season
        xSea.attrs['long_name'] = "seasonal means: " + xSea.attrs['long_name']
        xSea = xSea.rename("xSea")
        # Return the newly created DataArray
        return xSea
    
    
    # Calculating seasonal mean for rank = 4
    if (rank_of_dim == 4):
        
        # Calculate seasonal mean for each season
        dr = xMon.rolling(time = 3, center = True).mean(skipna = True)
        dr[0,:,:,:] = (xMon[0,:,:,:] + xMon[1,:,:,:]) * 0.5
        dr[(no_of_time-1),:,:,:] = (xMon[(no_of_time-2),:,:,:] + xMon[(no_of_time-1),:,:,:]) * 0.5
        
        # Create a new DataArray using the existing ones
        xSea = xMon.copy(data = dr)
        
        # Add and update the attributes
        xSea.attrs['season'] = season
        xSea.attrs['long_name'] = "seasonal means: " + xSea.attrs['long_name']
        xSea = xSea.rename("xSea")
        # Return the newly created DataArray
        return xSea

In [9]:
xSea = month_to_season12(sst)

In [10]:
xSea

<xarray.DataArray 'xSea' (time: 120, lat: 180, lon: 360)>
array([[[-1000.    , -1000.    , ..., -1000.    , -1000.    ],
        [-1000.    , -1000.    , ..., -1000.    , -1000.    ],
        ...,
        [       nan,        nan, ...,        nan,        nan],
        [       nan,        nan, ...,        nan,        nan]],

       [[-1000.    , -1000.    , ..., -1000.    , -1000.    ],
        [-1000.    , -1000.    , ..., -1000.    , -1000.    ],
        ...,
        [       nan,        nan, ...,        nan,        nan],
        [       nan,        nan, ...,        nan,        nan]],

       ...,

       [[-1000.    , -1000.    , ..., -1000.    , -1000.    ],
        [ -667.2667,  -667.2667, ...,  -667.2667,  -667.2667],
        ...,
        [       nan,        nan, ...,        nan,        nan],
        [       nan,        nan, ...,        nan,        nan]],

       [[-1000.    , -1000.    , ..., -1000.    , -1000.    ],
        [-1000.    , -1000.    , ..., -1000.    , -1000.    ],
  